# Setting up the RAG Pipeline using Cortex Search

In this notebook, we're setting up the RAG pipeline using the Avalanche customer review data from the previously prepared data stored in the stage. 

By the end of the tutorial, we'll have a RAG pipeline ready to allow us to ask any questions about the data.

## List staged data

Here, we're listing the contents of the staged data at `@avalanche.customer_reviews`

In [ ]:
ls @avalanche.customer_reviews;

## Parse content

Here, we're extracting or parsing content from the `DOCX` files stored in stage by leveraging the Cortex `PARSE_DOCUMENT()` function.

The parsed content will be stored at a newly created table at  `AVALANCHE.AVALANCHE.PARSED_CONTENT`.

In [ ]:
CREATE OR REPLACE TABLE AVALANCHE.AVALANCHE.PARSED_CONTENT AS SELECT 
      relative_path,
      TO_VARCHAR(
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
          @avalanche.customer_reviews, 
          relative_path, 
          {'mode': 'LAYOUT'}
        ) :content
      ) AS parsed_text
    FROM directory(@avalanche.customer_reviews)
    WHERE relative_path LIKE '%.docx'

In [ ]:
SELECT * FROM AVALANCHE.AVALANCHE.PARSED_CONTENT LIMIT 5


In [ ]:
CREATE OR REPLACE TABLE AVALANCHE.AVALANCHE.CHUNKED_CONTENT (
    file_name VARCHAR,
    CHUNK VARCHAR
);

INSERT INTO AVALANCHE.AVALANCHE.CHUNKED_CONTENT (file_name, CHUNK)
SELECT
    relative_path,
    c.value AS CHUNK
FROM
    AVALANCHE.AVALANCHE.PARSED_CONTENT,
    LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
        parsed_text,
        'markdown',
        1800,
        250
    )) c;

In [ ]:
SELECT * FROM AVALANCHE.AVALANCHE.CHUNKED_CONTENT LIMIT 10

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE AVALANCHE.AVALANCHE.AVALANCHE_SEARCH_SERVICE
    ON chunk
    WAREHOUSE = chanin_xs
    TARGET_LAG = '1 minute'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
    SELECT
        file_name,
        chunk
    FROM AVALANCHE.AVALANCHE.CHUNKED_CONTENT
    );

In [ ]:

--query it with SQL
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'AVALANCHE.AVALANCHE.AVALANCHE_SEARCH_SERVICE',
      '{
         "query": "Any goggles review?",
         "columns":[
            "file_name",
            "CHUNK"
         ],
         "limit":3
      }'
  )
)['results'] as results;

In [ ]:

#query it with Python
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session
import streamlit as st
import json
import pandas as pd

session = get_active_session()

prompt="Any goggles review?"

root = Root(session)

# query service
svc = (root
  .databases["AVALANCHE"]
  .schemas["AVALANCHE"]
  .cortex_search_services["AVALANCHE_SEARCH_SERVICE"]
)

resp = svc.search(
  query=prompt,
  columns=["CHUNK", "file_name"],
  limit=3
).to_json()

#optional - I just like the way this looks...
json_conv = json.loads(resp) if isinstance(resp, str) else resp
search_df = pd.json_normalize(json_conv['results'])

#st.write(search_df)
for _, row in search_df.iterrows():
    st.write(f"**{row['CHUNK']}**")
    st.caption(row['file_name'])
    st.write('---')